# August 17th, 2018
## Benjamin Adam Catching
## Polio-virus DIPs Project
## Andino Lab

Code has now been integrated into the Microfluidics_analysis file, now apply it to all cells of the conditions:  
* P1 and DIP in Optiprep (condition 1)  
* PI and DIP not in Optiprep (condition 2)  
* Just DIP in Optiprep (condition 3)

In [1]:
# Import necessary packages
import glob
import numpy as np
import matplotlib.pyplot as plt
import skimage.io
import seaborn as sns
import skimage
import skimage.filters
import skimage.measure
import skimage.segmentation
import skimage.feature
import scipy.ndimage as ndi
import scipy
import pandas as pd
import os